In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_events= pd.read_csv('/Users/wassim/threejs-cd-preview/ressources/Dark_GD_Contest_Events.csv')
df_edges= pd.read_csv('/Users/wassim/threejs-cd-preview/ressources/Dark_GD_Contest_Edges.csv')
# print(df_events.head())
# print(df_edges.info())

# df_events["date"] = df_events["date"].dt.year


df_events = df_events.dropna()
df_events = df_events.drop_duplicates()
df_edges.head()
df_edges = df_edges.dropna()
df_edges = df_edges.drop_duplicates()


# entrypoints = df_edges ["target"].unique()

source = df_edges["Source"]
target = df_edges["Target"]



print(prdecessors)
print(prdecessors.info())





 


2      False
17     False
18     False
20     False
22     False
       ...  
555    False
556    False
601    False
622    False
624    False
Name: Target, Length: 74, dtype: bool
<class 'pandas.core.series.Series'>
Index: 74 entries, 2 to 624
Series name: Target
Non-Null Count  Dtype
--------------  -----
74 non-null     bool 
dtypes: bool(1)
memory usage: 666.0 bytes
None
